In [4]:
import pandas as pd
import os
import glob

inputPath = '../data/records_samples/'

# 1. Load dataframe with records
Create records_samples folder in data if not there
Place records.pkl and samples.pkl in folder from shared folder in google drive

In [5]:
df_record = pd.read_pickle(os.path.join(inputPath,'records.pkl'))
df_record.head()

,Id,nsamples,Accession,ExtRelations,FTPLink,GDS,GEO2R,GPL,GSE,Item,...,entryType,gdsType,n_samples,ptechType,subsetInfo,summary,suppFile,taxon,title,valType
0,200000001,38.0,GSE1,[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSEnnn/G...,,yes,7,1,[],...,GSE,Expression profiling by array,38,,,This series represents a group of cutaneous ma...,,Homo sapiens,NHGRI_Melanoma_class,
1,200000003,342.0,GSE3,[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSEnnn/G...,,yes,10;9,3,[],...,GSE,Expression profiling by array,342,,,We investigated the changes in gene expression...,,Homo sapiens,Renal Cell Carcinoma Differential Expression,
2,200000010,4.0,GSE10,[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSEnnn/G...,,no,4,10,[],...,GSE,Expression profiling by SAGE,4,,,Human retinal and RPE SAGE libraries. Profile ...,,Homo sapiens,eye-SAGE,
3,200000014,765.0,GSE14,[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSEnnn/G...,,no,4;1349;1485;11,14,[],...,GSE,Expression profiling by SAGE,765,,,This series represents the Cancer Genome Anato...,,Mus musculus; Homo sapiens,Cancer Genome Anatomy Project (CGAP) SAGE Libr...,
4,200000016,25.0,GSE16,[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSEnnn/G...,,yes,28,16,[],...,GSE,Genome variation profiling by genome tiling array,25,,,This series represents the data set from the p...,,Homo sapiens,Microarray-based CGH in human cancer cell lines,


# 2. Load DNorm input data
First part of notebook: we will create the input data for dnorm 
From a pandas dataframe that contains the records (records.pkl)
Input whether you want all records or a certain number of records 
 - For all records: all_rows=1
 - For fixed number, all_rows=0 and n_rows=[number of rows you want]
 
This will output DNorm input data to data/DNorm_input.txt
 
To run DNorm you then place this into the DNorm folder and run code 
For DNorm instructions download java app here: ncbi.nlm.nih.gov/research/bionlp/Tools/dnorm and look at readme text file inside folder)

In [ ]:
# this cell creates a tab delimited file for the DNorm demo ./runDNorm

all_rows = 1
n_rows = 1000


if all_rows==1:
    n_rows = len(df_record)

rec4input = df_record.loc[:n_rows,:]

outF = open(os.path.join(inputPath, 'dnorm_input.txt'), "w")
# build list of strings
rec4input = rec4input.reset_index()
for i in range(len(rec4input)):
    Id = rec4input.loc[i,:].Id
    title = rec4input.loc[i,:].title
    summary = rec4input.loc[i,:].summary
    outF.write(Id)
    outF.write('\t')
    outF.write(title +'. '+ summary)
    outF.write("\n")

# 3. Load input for DNorm with metathesaurus

In [8]:
# This cell creates input in the form of
# 10192393|t|A common human skin tumour is caused by activating mutations in beta-catenin.
# 10192393|a|WNT signalling orchestrates a number of developmental programs. In response to this stimulus, cytoplasmic beta-catenin (encoded by CTNNB1) is stabilized, enabling downs$
# for the more advanced function ./ApplyDNorm

all_rows = 1
n_rows = 10

if all_rows==1:
    n_rows = len(df_record)

rec4input = df_record.loc[:n_rows,:]

outF = open(os.path.join(inputPath, 'dnorm_input_v2.txt'), "w")
# build list of strings
rec4input = rec4input.reset_index()
for i in range(len(rec4input)):
    Id = rec4input.loc[i,:].Id
    title = rec4input.loc[i,:].title.replace('|','')
    summary = rec4input.loc[i,:].summary.replace('|','')
    # remove | characters from title and summary
    outF.write(Id+'|t|'+title)
    outF.write("\n")
    outF.write(Id+'|a|'+summary)
    outF.write("\n")


# 4. Turn DNorm output into pandas df
 - After running DNorm, take text output file and paste into output_data_path folder
 - Turn into dataframe and save to output path

In [ ]:
# this cell takes the output from the demo 

output_data_path = '../data/DNorm/'

if os.path.isdir(output_data_path)!=1:
    os.mkdir(output_data_path)

output_file_path = os.path.join(output_data_path, 'record_tags.txt')

with open(output_file_path,'r') as f:
    data = f.read().splitlines()
    
table = [d.split('\t') for d in data]
headers = ['Id', 'start', 'end', 'disease_tag', 'ONTid']
df = pd.DataFrame(table, columns=headers)

s = df.ONTid.str.split(':')
df_aux = pd.DataFrame.from_items(zip(s.index, s.values)).T
df_aux.columns = ['ont', 'unique_id']
df['ont'] = df_aux['ont']
df['unique_id'] = df_aux['unique_id']
df = df.drop('ONTid', axis = 1)

df.to_pickle(os.path.join(output_data_path,'disease_tags_demo.pkl'))

In [32]:
#this cell takes the output from the more advanced function ./applyDNorm

output_data_path = '../data/DNorm/'

if os.path.isdir(output_data_path)!=1:
    os.mkdir(output_data_path)

output_file_path = os.path.join(output_data_path, 'dnorm_output_v2.txt')

with open(output_file_path,'r') as f:
    data = f.read().splitlines()
    
table = [d.split('\t') for d in data if len(d.split('\t'))>1]
headers = ['Id', 'start', 'end', 'disease_tag', 'tag_type', 'ONTid']
df = pd.DataFrame(table, columns=headers)

s = df.ONTid.str.split(':')
df_aux = pd.DataFrame.from_items(zip(s.index, s.values)).T
df_aux.columns = ['ont', 'unique_id']
df['ont'] = df_aux['ont']
df['unique_id'] = df_aux['unique_id']
df = df.drop('ONTid', axis = 1)

df = pd.merge(df_record[['Id', 'PDAT']], df, on='Id', how='outer')

df.to_pickle(os.path.join(output_data_path,'disease_tags_dnorm_advanced.pkl'))

df

/Users/clairechambers/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.


,Id,PDAT,start,end,disease_tag,tag_type,ont,unique_id
0,200000001,2001/01/22,55,84,cutaneous malignant melanomas,Disease,OMIM,155600
1,200000003,2001/07/19,0,20,Renal Cell Carcinoma,Disease,MESH,D002292
2,200000003,2001/07/19,140,153,kidney cancer,Disease,MESH,D007680
3,200000003,2001/07/19,655,667,renal cancer,Disease,MESH,D007680
4,200000003,2001/07/19,817,837,renal cell carcinoma,Disease,MESH,D002292
5,200000003,2001/07/19,869,881,kidney tumor,Disease,MESH,D007674
6,200000003,2001/07/19,1055,1072,epithelial tumors,Disease,MESH,D002277
7,200000003,2001/07/19,1108,1128,renal cell carcinoma,Disease,MESH,D002292
8,200000003,2001/07/19,1284,1310,renal clear cell carcinoma,Disease,MESH,D002292
9,200000010,2001/10/03,NaN,NaN,NaN,NaN,NaN,NaN
